In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
df=pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv",encoding="latin",header=None,on_bad_lines="skip")

In [3]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df.rename(columns={0:"labels",5:"tweets"},inplace=True)
df.drop([1,2,3,4],axis=1,inplace=True)

In [5]:
df

,labels,tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [6]:
df.isnull().sum()

labels    0
tweets    0
dtype: int64

In [7]:
df['labels'] = df['labels'].replace(4,1)

In [8]:
#shuffle the dataset
df=df.sample(frac=1)
df.head()

,labels,tweets
311189,0,I can't go on the comp. My mouse broken.
292729,0,i just washed my car and of course it rained
275457,0,its going slow so slow learning
461574,0,I shouldn't be up reading tweets...I have to g...
912675,1,Home at last HAPPY MOTHERS TO ALL THE MOTHERS ...


In [9]:
df=df.head(20000)
df.head()

,labels,tweets
311189,0,I can't go on the comp. My mouse broken.
292729,0,i just washed my car and of course it rained
275457,0,its going slow so slow learning
461574,0,I shouldn't be up reading tweets...I have to g...
912675,1,Home at last HAPPY MOTHERS TO ALL THE MOTHERS ...


In [10]:
df['tweets'].values[0]

"I can't go on the comp. My mouse broken. "

In [11]:
import re
import spacy
import nltk
from nltk.corpus import stopwords
nlp=spacy.load('en_core_web_sm')
nltk.download('stopwords')

stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def preprocessed_tweet(tweet):
    tweet=tweet.lower()
    tweet=re.sub(r"http\S+|www|S+|https\S+",'',tweet,flags=re.MULTILINE)
    tweet=re.sub(r"@\w+",'',tweet)
    tweet=re.sub(r"^w\s#",'',tweet)

    tokens=[token.lemma_ for token in nlp(tweet)]
    tokens=[token for token in tokens if token not in stop_words]
    tokens=[token for token in tokens if len(token)>1]
    preprocessed_tweet=' '.join(tokens)
    
    
    return preprocessed_tweet
    

In [13]:
df['clean_tweets']=[preprocessed_tweet(tw) for tw in df['tweets']]

In [14]:
df.head()

,labels,tweets,clean_tweets
311189,0,I can't go on the comp. My mouse broken.,go comp mouse break
292729,0,i just washed my car and of course it rained,wash car course rain
275457,0,its going slow so slow learning,go slow slow learn
461574,0,I shouldn't be up reading tweets...I have to g...,read tweet ... get early agileroot ... get
912675,1,Home at last HAPPY MOTHERS TO ALL THE MOTHERS ...,home last happy mother mother special thank fa...


In [15]:
def get_word_len(tweet):
    return len(tweet.split(' '))

def get_char_len(tweet):
    return len(tweet)

In [16]:
df['tweet_word_len']=df['clean_tweets'].apply(get_word_len)
df['tweet_char_len']=df['clean_tweets'].apply(get_char_len)

In [17]:
df.head()

,labels,tweets,clean_tweets,tweet_word_len,tweet_char_len
311189,0,I can't go on the comp. My mouse broken.,go comp mouse break,4,19
292729,0,i just washed my car and of course it rained,wash car course rain,4,20
275457,0,its going slow so slow learning,go slow slow learn,4,18
461574,0,I shouldn't be up reading tweets...I have to g...,read tweet ... get early agileroot ... get,8,42
912675,1,Home at last HAPPY MOTHERS TO ALL THE MOTHERS ...,home last happy mother mother special thank fa...,15,91


In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as ps
import plotly.figure_factory as ff
from plotly.subplots import make_subplots



In [19]:
positive_counts=len(df[df['labels']==1])
negative_counts=len(df[df['labels']==0])

In [20]:
fig=ps.pie(values=[positive_counts,negative_counts],title="Distribution of Sentimenets of tweets",template='simple_white')
fig.show()

In [21]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(df['clean_tweets'],df['labels'],test_size=0.2,random_state=42)

In [22]:
x_test,x_val,y_test,y_val=train_test_split(xtest,ytest,test_size=0.5)


In [23]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer


In [24]:
vec=CountVectorizer(binary=True)
vec=vec.fit(df['clean_tweets'])
enc_tweets=vec.fit_transform(df['clean_tweets']).toarray()

In [25]:
xtrain,xtest,ytrain,ytest=train_test_split(enc_tweets,df['labels'],test_size=0.2,random_state=42)

In [26]:
x_test,x_val,y_test,y_val=train_test_split(xtest,ytest,test_size=0.5)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [28]:
xtrain.shape

(16000, 17926)

In [29]:
xtest.shape

(4000, 17926)

In [30]:
xtrain[0].sum()

7

In [31]:
enc_tweets.shape[1]

17926

In [32]:
model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(enc_tweets.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │     9,178,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,248,449 (35.28 MB)

 Trainable params: 9,248,449 (35.28 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
epochs=10
batch_size=32

In [35]:
history_1=model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(x_val,y_val))

Epoch 1/10


I0000 00:00:1732375564.511998      71 service.cc:145] XLA service 0x7829e8005280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732375564.512078      71 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1732375564.512085      71 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 37/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4996 - loss: 0.6910

I0000 00:00:1732375565.925049      71 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6526 - loss: 0.6114 - val_accuracy: 0.7460 - val_loss: 0.5192
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8648 - loss: 0.3220 - val_accuracy: 0.7255 - val_loss: 0.5845
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9554 - loss: 0.1256 - val_accuracy: 0.7160 - val_loss: 0.8799
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9849 - loss: 0.0406 - val_accuracy: 0.7220 - val_loss: 1.2464
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9919 - loss: 0.0214 - val_accuracy: 0.7145 - val_loss: 1.5128
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9926 - loss: 0.0181 - val_accuracy: 0.7190 - val_loss: 1.5669
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9939 - loss: 0.0128 - val_accuracy: 0.7175 - val_loss: 1.6329
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9947 - loss: 0.0107 - val_accuracy: 0.7145 - val_

In [36]:
model.evaluate(xtest,ytest)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7160 - loss: 1.9855


[1.9579740762710571, 0.7124999761581421]

In [37]:
vec=CountVectorizer(binary=True)
vec=vec.fit(df['clean_tweets'])
enc_tweets=vec.fit_transform(df['clean_tweets']).toarray()
xtrain,xtest,ytrain,ytest=train_test_split(enc_tweets,df['labels'],test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(xtest,ytest,test_size=0.5)

model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(enc_tweets.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history_1=model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(x_val,y_val))
model.evaluate(xtest,ytest)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6668 - loss: 0.6076 - val_accuracy: 0.7170 - val_loss: 0.5446
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8609 - loss: 0.3255 - val_accuracy: 0.7175 - val_loss: 0.6418
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9498 - loss: 0.1266 - val_accuracy: 0.7080 - val_loss: 0.9891
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9839 - loss: 0.0390 - val_accuracy: 0.7140 - val_loss: 1.3422
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9895 - loss: 0.0237 - val_accuracy: 0.7180 - val_loss: 1.5655
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9931 - loss: 0.0157 - val_accuracy: 0.7175 - val_loss: 1.5823
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9936 - loss: 0.0150 - val_accuracy: 0.7165 - val_loss: 1.6828
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9949 - loss: 0.0105 - val_accuracy: 0.

[2.119734525680542, 0.7102500200271606]

In [38]:
tfidf=TfidfVectorizer()

enc_tweets=tfidf.fit_transform(df['clean_tweets']).toarray()
xtrain,xtest,ytrain,ytest=train_test_split(enc_tweets,df['labels'],test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(xtest,ytest,test_size=0.5)

model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(enc_tweets.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history_1=model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(x_val,y_val))
model.evaluate(xtest,ytest)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6359 - loss: 0.6125 - val_accuracy: 0.7340 - val_loss: 0.5382
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8608 - loss: 0.3291 - val_accuracy: 0.7095 - val_loss: 0.6226
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9400 - loss: 0.1518 - val_accuracy: 0.7015 - val_loss: 0.8234
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9835 - loss: 0.0468 - val_accuracy: 0.6785 - val_loss: 1.0725
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9905 - loss: 0.0241 - val_accuracy: 0.7000 - val_loss: 1.2416
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9930 - loss: 0.0146 - val_accuracy: 0.6845 - val_loss: 1.3119
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9947 - loss: 0.0131 - val_accuracy: 0.6905 - val_loss: 1.3642
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9946 - loss: 0.0112 - val_accuracy: 0.

[1.5529597997665405, 0.703000009059906]

In [39]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [40]:
seq_len=20
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['clean_tweets'])
word_index=tokenizer.word_index
index_word=tokenizer.index_word
sequences=tokenizer.texts_to_sequences(df['clean_tweets'])
enc_tweets=pad_sequences(sequences,maxlen=seq_len,padding='post')

In [41]:
enc_tweets.shape

(20000, 20)

In [42]:
enc_tweets[0]

array([   1, 1260, 1939,  146,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [43]:
len(word_index)

18155

In [44]:
xtrain,xtest,ytrain,ytest=train_test_split(enc_tweets,df['labels'],test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(xtest,ytest,test_size=0.5)

In [45]:
model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(seq_len,)))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
history_4=model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(x_val,y_val))

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.4882 - loss: 83.0706 - val_accuracy: 0.5115 - val_loss: 0.7068
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5014 - loss: 4.2683 - val_accuracy: 0.4960 - val_loss: 0.6937
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5160 - loss: 1.1394 - val_accuracy: 0.4915 - val_loss: 0.6934
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5029 - loss: 0.8471 - val_accuracy: 0.4915 - val_loss: 0.6934
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4970 - loss: 0.7811 - val_accuracy: 0.4915 - val_loss: 0.6933
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5106 - loss: 0.7456 - val_accuracy: 0.5085 - val_loss: 0.6931
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5027 - loss: 0.7147 - val_accuracy: 0.5085 - val_loss: 0.6931
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4943 - loss: 0.7035 - val_accuracy: 0

In [47]:
from tensorflow.keras.layers import Embedding,Flatten
embedding_len=50
model=Sequential()
model.add(Embedding(input_dim=len(word_index)+1,output_dim=embedding_len,input_length=seq_len))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history5=model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(x_val,y_val))


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning:

Argument `input_length` is deprecated. Just remove it.



500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5849 - loss: 0.6489 - val_accuracy: 0.7430 - val_loss: 0.5240
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8538 - loss: 0.3578 - val_accuracy: 0.7025 - val_loss: 0.6329
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9509 - loss: 0.1361 - val_accuracy: 0.6785 - val_loss: 0.8394
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9765 - loss: 0.0658 - val_accuracy: 0.6735 - val_loss: 1.0363
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9842 - loss: 0.0431 - val_accuracy: 0.6745 - val_loss: 1.2161
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9867 - loss: 0.0358 - val_accuracy: 0.6725 - val_loss: 1.3926
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9882 - loss: 0.0287 - val_accuracy: 0.6645 - val_loss: 1.5299
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9887 - loss: 0.0261 - val_accuracy: 0.6640 - val_